In [1]:
import scipy as sp
import numpy as np
import os 
import pickle

import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime

from architectures.cycleGAN import *

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
def insert(X_inner, X_outer):
    X=X_outer
    
    for i in range(26,78):
        for j in range(32,96):
            X[i,j]=X_inner[i-26,j-36]
    
    return X

In [ ]:
# some constants

LEARNING_RATE_D = 0.0001
LEARNING_RATE_G = 0.0001
BETA1 = 0.5
BATCH_SIZE = 128
EPOCHS = 1
SAVE_SAMPLE_PERIOD = 100
#task='TRAIN'
task='TEST'
SEED=1

PATH = 'cycleGAN_test'

In [ ]:
trunc_normal= tf.truncated_normal_initializer(stddev=0.02)
normal = tf.random_normal_initializer(stddev=0.02)

uniform = tf.random_uniform_initializer()
glorot_uniform = tf.glorot_uniform_initializer()
glorot_normal = tf.glorot_normal_initializer()

global d_sizes, g_sizes

d_sizes_A = {
         'conv_layers': [(2, 6, 2, False, 1, lrelu, trunc_normal),
                         (64, 6, 2, True, 1, lrelu, trunc_normal)],
         'dense_layers': [(1024, True, 1, lrelu ,normal)],
         'readout_layer_w_init':normal,
}

d_sizes_B = {
         'conv_layers': [(2, 6, 2, False, 1, lrelu, trunc_normal),
                         (64, 6, 2, True, 1, lrelu, trunc_normal)],
         'dense_layers': [(1024, True, 1, lrelu ,normal)],
         'readout_layer_w_init':normal,
}

g_sizes_A={
         'conv_layer_0':[(4, 8, 2, False, 1, tf.nn.relu, normal)],
         'conv_layer_1':[(8, 8, 2, False, 1, tf.nn.relu, normal)],
         'convblock_layer_0':[(16, 4, 2, True, 1, tf.nn.relu, normal),
                                (16, 4, 1, False, 1, tf.nn.relu, normal),
                                (16, 4, 2, True, 1, tf.nn.relu, normal)],
         'convblock_shortcut_layer_0':[(16, 1, 4, False, 1, normal)],
         'deconv_layer_0':[(8, 8, 2, False, 1, tf.nn.relu, normal)],
         'deconv_layer_1':[(1, 8, 2, False, 1, tf.nn.relu, normal)],
         
        
         
}

g_sizes_B={
         'conv_layer_0':[(4, 8, 2, False, 1, tf.nn.relu, normal)],
         'conv_layer_1':[(8, 8, 2, False, 1, tf.nn.relu, normal)],
         'convblock_layer_0':[(16, 4, 2, True, 1, tf.nn.relu, normal),
                                (16, 4, 1, False, 1, tf.nn.relu, normal),
                                (16, 4, 2, True, 1, tf.nn.relu, normal)],
         'convblock_shortcut_layer_0':[(16, 1, 4, False, 1, normal)],
         'deconv_layer_0':[(8, 8, 2, False, 1, tf.nn.relu, normal)],
         'deconv_layer_1':[(1, 8, 2, False, 1, tf.nn.relu, normal)],
        
         
}



In [ ]:
def HCAL():

    with open('/data/dlanci/samples/reco/sample0.pickle', 'rb') as f:
        reco=pickle.load(f)
    #with open('samples/reco/sample1.pickle', 'rb') as f:
    #    reco1=pickle.load(f)
        
    #reco=np.concatenate((reco,reco1),axis=0)
    
    pos=[]
    for i in range(len(reco)):
        if np.array_equal(reco[i],np.zeros_like(reco[i])):
            pos.append(i)
    reco = np.delete(reco, pos, axis=0)
    reco[reco!=0]/=np.max(reco)
    X_train_B=reco
    
    with open('/data/dlanci/samples/true/sample0.pickle', 'rb') as f:
        true=pickle.load(f)
    true = true[:,1:true.shape[1]-1,1:true.shape[2]-1]
    #with open('samples/true/sample1.pickle', 'rb') as f:
    #    true1=pickle.load(f)
        
    #true=np.concatenate((true,true1),axis=0)

    true = np.delete(true, pos, axis=0)
    true[true!=0]/=np.max(true)
    
    X_train_A = true
    
    m, n_H, n_W, n_C = X_train_A.shape
    
    #test_size = 5000
    #train_size = m - test_size
    
    out = np.zeros_like(X_train_A)
    
    for i in range(len(out)):
        out[i] = insert(X_train_B[i],out[i])
    
    X_train_B = out
    
    tf.reset_default_graph()

    gan = cycleGAN(n_H, n_W, n_C, d_sizes_A, d_sizes_B, g_sizes_A, g_sizes_B,
                   lr_g=LEARNING_RATE_G, lr_d=LEARNING_RATE_D,beta1=BETA1,
                  batch_size=BATCH_SIZE, epochs=EPOCHS,
                  save_sample=SAVE_SAMPLE_PERIOD, path=PATH, seed=SEED)
    
    vars_to_train= tf.trainable_variables()
    
    if task == 'TRAIN':
        init_op = tf.global_variables_initializer()
        
    if task == 'TEST':
        vars_all = tf.global_variables()
        vars_to_init = list(set(vars_all)-set(vars_to_train))
        init_op = tf.variables_initializer(vars_to_init)
        
    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()    
        
    with tf.Session() as sess:
        
        sess.run(init_op)

        if task=='TRAIN':
            print('\n Training...')
            
            if os.path.exists(PATH+'/'+PATH+'.ckpt.index'):
                saver.restore(sess,PATH+'/'+PATH+'.ckpt')
                print('Model restored.')
            
            gan.set_session(sess)
            gan.fit(X_train_A, X_train_B)
            
            save_path = saver.save(sess, PATH+'/'+PATH+'.ckpt')
            print("Model saved in path: %s" % save_path)
        
        if task=='TEST':
            print('\n Evaluate model on test set...')
            saver.restore(sess,PATH+'/'+PATH+'.ckpt')
            print('Model restored.')
            
            gan.set_session(sess) 
            
        done = False
        while not done:
            
            
            j = np.random.choice(len(X_train_A))
            true_img = X_train_A[j]
            sample_img = gan.get_sample(true_img)
            
            plt.subplot(1,2,1)
            plt.imshow(true_img.reshape(n_H,n_W),cmap='gray')
            plt.subplot(1,2,2)
            plt.imshow(sample_img.reshape(n_H,n_W),cmap='gray')
            
            fig=plt.gcf()
            fig.set_size_inches(5,8)
            plt.savefig(PATH+'/sample_{0}_at_iter_{1}.png'.format(j, total_iters),dpi=300)
            
            ans = input("Generate another?")
            if ans and ans[0] in ('n' or 'N'):
                done = True

In [ ]:
if __name__=='__main__':

    if task == 'TRAIN':
        if not os.path.exists(PATH):
            os.mkdir(PATH)
    
        elif os.path.exists(PATH):
            if os.path.exists(PATH+'/checkpoint'):
                ans = input('A previous checkpoint already exists, choose the action to perform \n \n 1) Overwrite the current model saved at '+PATH+'/checkpoint \n 2) Start training a new model \n 3) Restore and continue training the previous model \n ')
                
                if ans == '1':
                    print('Overwriting existing model in '+PATH)
                    for file in os.listdir(PATH):
                        file_path = os.path.join(PATH, file)
                        try:
                            if os.path.isfile(file_path):
                                os.unlink(file_path)
                            #elif os.path.isdir(file_path): shutil.rmtree(file_path)
                        except Exception as e:
                            print(e)
                            
                elif ans == '2':
                    PATH = input('Specify the name of the model, a new directory will be created.\n')
                    os.mkdir(PATH)    
        
        HCAL()
   
    elif task == 'TEST': 
        if not os.path.exists(PATH+'/checkpoint'):
            print('No checkpoint to test')
        else:
            HCAL()